In [ ]:
import torch 
import torchvision
import torch.nn as nn 
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder
from tqdm import tqdm

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
!nvidia-smi

In [ ]:
data_transform = transforms.Compose([
    transforms.Resize(size=(256, 256)),
    transforms.Grayscale(1),
    transforms.RandomHorizontalFlip(p = 0.5),
    transforms.ToTensor()
])

In [ ]:
path = Path('###########')

In [ ]:
train_dir = path / 'train'
test_dir = path /'test'

In [ ]:
train_dir, test_dir

In [ ]:
trainFolder = ImageFolder(train_dir, data_transform, target_transform=None)
testFolder = ImageFolder(test_dir, data_transform, target_transform=None)

In [ ]:
len(trainFolder), len(testFolder)

In [ ]:
class_id = trainFolder.class_to_idx
class_dict = trainFolder.classes

class_id, class_dict

In [ ]:
import os

In [ ]:
trainData = DataLoader(trainFolder, shuffle=True, num_workers= os.cpu_count(), batch_size=4)
testData = DataLoader(testFolder, num_workers= os.cpu_count(), batch_size=4)

In [ ]:
class FaceModel(nn.Module):
    def __init__(self, inshape = 1, hidden_shape = 16, out_shape = 2):
        super(FaceModel, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(inshape, hidden_shape, kernel_size=2, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=1),
            nn.Dropout(0.2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(hidden_shape, hidden_shape, kernel_size=2, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=1),
            nn.Dropout(0.2)
        )
        self.layer3 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(hidden_shape * 256 * 256, out_shape)
        )


    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x


In [ ]:
model = FaceModel(inshape=1, hidden_shape=16, out_shape=2).to(device)

In [ ]:
img, label = next(iter(trainData))
plt.title(label=label[0])
plt.imshow(img[0].permute(1,2,0))
plt.plot()
img.shape

In [ ]:
pred = model(img.to(device))

In [ ]:
from torchmetrics import Accuracy

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=0.001)
acc = Accuracy('BINARY', num_classes=2)

In [ ]:
import cv2
from PIL import Image

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


In [ ]:
epochs = 10

for epoch in range(epochs):
    model.train()
    total_train_loss = 0
    for batch in tqdm(trainData):
        X,y = batch
        y_pred = model(X.to(device))
        optim.zero_grad()
        loss = loss_fn(y_pred.to(device), y.to(device))
        loss.backward()
        optim.step()
        total_train_loss += loss.item()
    total_train_loss /= len(trainData)


    model.eval()
    test_loss = 0
    test_acc = 0
    with torch.inference_mode():
        for batch in tqdm(testData):
            X, y = batch
            y_pred = model(X.to(device))
            loss = loss_fn(y_pred.to(device), y.to(device))
            test_loss += loss.item()
            test_acc += acc(y_pred.argmax(dim = 1).to('cpu'), y.to('cpu'))

        total_test_loss = test_loss / len(testData)
        total_test_Acc = test_acc / len(testData)

    print(f'Epoch: {epoch + 1}, trainLoss: {total_train_loss}, test_loss: {total_test_loss}, test Acc: {total_test_Acc}')

In [ ]:
torch.save(model.state_dict(), 'faceRecognition.pt')

In [ ]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert BGR frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Convert NumPy array to PIL Image
    pil_image = Image.fromarray(rgb_frame)
    
    # Apply transformations
    transformed_image = data_transform(pil_image)
    
    # Add batch dimension and make a prediction
    input_tensor = transformed_image.unsqueeze(0).to(device)  # Add batch dimension and move to device
    with torch.no_grad():
        y_pred = model(input_tensor)

    # Get the predicted class
    predicted_class = y_pred.argmax(dim=1).item()
    # Convert prediction to string for display
    y_pred_str = str(class_dict[predicted_class])
    
    # Display the prediction on the frame
    cv2.putText(frame, y_pred_str, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow('Face Recognition', frame)

    # Break the loop if the user presses 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()